TPCH Query 6 and Query 5 runs

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MyApp").getOrCreate()
sc = spark.sparkContext

In [ ]:
sc.range(1000).count()

In [ ]:
import os
from huggingface_hub import snapshot_download
#tpch_dataset = 'tpch-sfdot001g' # for sf=0.001g 
tpch_dataset = 'tpch-sf1g' # for sf=1g
local_dir=f'./{tpch_dataset}/'
if not os.path.exists(local_dir):
    print(f'Downloading dataset into {local_dir} ...')
    snapshot_download(
        repo_id="hkverma/"+tpch_dataset,
        repo_type="dataset",
        local_dir=local_dir,
        local_dir_use_symlinks=False  # ensures real copies, not symlinks
    )
    print('Download Complete.')
else:
    print(f"Dataset already exists in {local_dir}, skipping download.")  

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, DoubleType

In [ ]:
schema = StructType([StructField('L_ORDERKEY',IntegerType(),True),
                     StructField('L_PARTKEY',IntegerType(),True),
                     StructField('L_SUPPKEY',IntegerType(),True),
                     StructField('L_LINENUMBER',IntegerType(),True),
                     StructField('L_QUANTITY',IntegerType(),True),
                     StructField('L_EXTENDEDPRICE',DoubleType(),True),
                     StructField('L_DISCOUNT',DoubleType(),True),
                     StructField('L_TAX',DoubleType(),True),
                     StructField('L_RETURNFLAG',StringType(),True),
                     StructField('L_LINESTATUS',StringType(),True),
                     StructField('L_SHIPDATE',DateType(),True),
                     StructField('L_COMMITDATE',DateType(),True),
                     StructField('L_RECEIPTDATE',DateType(),True),
                     StructField('L_SHIPINSTRUCT',StringType(),True),
                     StructField('L_SHIPMODE',StringType(),True),
                     StructField('L_COMMENT',StringType(),True)])
schema

In [ ]:
lineitemDf = spark.read.format('csv').options(header='true').options(delimiter='|').schema(schema).load(f"{local_dir}/lineitem.csv")
lineitemDf.printSchema()
lineitemDf.show(5)
lineitemDf.createOrReplaceTempView("lineitem")

In [ ]:
sqlDf = spark.sql("select sum(l_extendedprice * l_discount) as revenue from lineitem "
                  "where l_shipdate >= date '1997-01-01' "
                  "and l_shipdate < date '1997-01-01' + interval '1' year "
                  " and l_discount between 0.07 - 0.01 and 0.07 + 0.01 "
                  " and l_quantity < 25;")
sqlDf.show()

Following neews to be done for the sql code shown below. load customer, orders, lineitem, supplier, nation, region tables. Then run query 5.

In [ ]:
customerDf = spark.read.format('csv').options(header='true').options(delimiter='|').options(inferSchema='true').load(f"{local_dir}/customer.csv")
customerDf.printSchema()
customerDf.show(5)
customerDf.createOrReplaceTempView("customer")

In [ ]:
ordersDf = spark.read.format('csv').options(header='true').options(delimiter='|').options(inferSchema='true').load(f"{local_dir}/orders.csv")
ordersDf.printSchema()
ordersDf.show(5)
ordersDf.createOrReplaceTempView("orders")

In [ ]:
supplierDf = spark.read.format('csv').options(header='true').options(delimiter='|').options(inferSchema='true').load(f"{local_dir}/supplier.csv")
supplierDf.printSchema()
supplierDf.show(5)
supplierDf.createOrReplaceTempView("supplier")

In [ ]:
nationDf = spark.read.format('csv').options(header='true').options(delimiter='|').options(inferSchema='true').load(f"{local_dir}/nation.csv")
nationDf.printSchema()
nationDf.show(5)
nationDf.createOrReplaceTempView("nation")

In [ ]:
regionDf = spark.read.format('csv').options(header='true').options(delimiter='|').options(inferSchema='true').load(f"{local_dir}/region.csv")
regionDf.printSchema()
regionDf.show(5)
regionDf.createOrReplaceTempView("region")

In [ ]:
sql5Df = spark.sql("select n_name, sum(l_extendedprice * (1 - l_discount)) as revenue "
    "from customer, orders, lineitem, supplier, nation, region "
    "where c_custkey = o_custkey "
    "and l_orderkey = o_orderkey "
    "and l_suppkey = s_suppkey "
    "and c_nationkey = s_nationkey "
    "and s_nationkey = n_nationkey "
    "and n_regionkey = r_regionkey "
    "and r_name = 'EUROPE' "
    "and o_orderdate >= date '1995-01-01' "
    "and o_orderdate < date '1995-01-01' + interval '1' year "
    "group by n_name order by revenue desc")
sql5Df.show()